In [96]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from konlpy.tag import Mecab
from tqdm import tqdm
from collections import Counter
import itertools # 2차원 배열 1차원 배열로
from sklearn.feature_extraction.text import TfidfVectorizer
import os #폴더 이름 가져오기

## 폴더 가져오기

In [126]:
# 고점 or 저점 파일을 파라미터 값으로 넣어 지정돤 폴더에서 csv파일 리스트들을 가져오는 함수
# 파라미터 : 절대경로에 있는 파일
def choice_file(filepath):
    file_list=os.listdir(filepath)
    cnt=0
    for i in file_list:
        i=filepath+"/"+i
        file_list[cnt]=i
        cnt+=1
    return file_list

file_list=choice_file('/Users/home/고점')
file_list

['/Users/home/고점/2022.08.13.csv',
 '/Users/home/고점/2022.02.15,16.csv',
 '/Users/home/고점/2022.03.01,02.csv',
 '/Users/home/고점/2022.11.30.csv',
 '/Users/home/고점/2021.10.20.csv',
 '/Users/home/고점/2022.10.04.csv',
 '/Users/home/고점/2022.05.31.csv',
 '/Users/home/고점/2022.04.04,05.csv',
 '/Users/home/고점/2022.09.12.csv',
 '/Users/home/고점/.ipynb_checkpoints',
 '/Users/home/고점/2022.11.05.csv']

## 필요한 데이터로 변경 (리스트) : 결측치 제거/ column 조정

In [127]:
files=[]
for i in file_list:
    try:
        df=pd.read_csv(i,encoding="UTF-8") # 크롤링한 파일 가져오기
        files.append(df)
    except:
        pass #.ipynb_checkpoints 파일때문

In [128]:
def delete(files):
    result=[]
    for df in files:
        df=df.replace("- dc official App","") # 글 내용없을때 들어가는 "- dc official App" 대체
        df=df.fillna('') # 결측치 제거
        df['data']=df['title']+ " "+df['content'] # 제목+내용을 하나로 합쳐서 'data'로 저장 
        del df['title'],df['content'],df['no'] # 제목,내용,글번호 삭제
        data=df['data']
        result.append(df)
    return result

files=delete(files)
files[0]

,date,data
0,2022.08.13 00:01:35,앵커에 다시 다박고 잔다 내일이면 100원은 가겠지??
1,2022.08.13 00:01:44,앵커는 20퍼는 더떨어져. 선물 숏때매 확 못내릴뿐. ㅇㅇ
2,2022.08.13 00:03:18,빗썸도 잡코펌핑좀 하나 할때됬는데
3,2022.08.13 00:04:02,롱 왤케 시원찮냐 ㅋㅋ흠
4,2022.08.13 00:04:23,12시 넘엇네
...,...,...
2360,2022.08.14 08:56:38,경주마 “하이브”
2361,2022.08.14 08:57:08,이클 ㅈ된거같은데 ㅋㅋㅋㅋㅋㅋ 지금이라도 팔아라 ㅋㅋㅋㅋ
2362,2022.08.14 08:57:50,시발 병신들아 코인 좀 떨어졌다고 울거나 그러는거 아니지? 잠깐 눈물좀 닦고
2363,2022.08.14 08:57:52,빗썸 콜라 다시 쏘냐


In [129]:
# 기본적인 전처리 과정을 진행하는 함수 
# 파라미터: pandas데이터프레임의 전처리를 원하는 컬럼
def data_processing(files):
    for file in files:
        file['data']=file['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규화
        file['data']=file['data'].str.replace('^ +', "") # 공백은 empty값으로 변경
        file['data'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
        file['data'].dropna(how='any') # Null 값 제거

data_processing(files)

/var/folders/mf/zx0plt5520l4g1dfsl85d0gm0000gn/T/ipykernel_30992/3334754670.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  file['data']=file['data'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규화
/var/folders/mf/zx0plt5520l4g1dfsl85d0gm0000gn/T/ipykernel_30992/3334754670.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  file['data']=file['data'].str.replace('^ +', "") # 공백은 empty값으로 변경


## test- 불용어 파일 리스트로 저장

In [130]:
f=open('stopwords.txt',encoding='utf-8') # 불용어 파일 
words=[]

for i in f:
    words.append(i)
f.close()

words=str(words)[2:-2]

stopwords=words.split(' ')

In [131]:
data=files

In [144]:
data[1]

,date,data
0,2022.02.15 00:00:12,비트스캠 음전해 온톨장군길막지마라 ㅋ
1,2022.02.15 00:00:18,형님들 보통 비트코인 몇시부터 몇시에 많이 하세요 핫한 시간대가 궁금합니다
2,2022.02.15 00:00:23,좋은 자리에 숏 하나는 사실 매우매우 큰 의미가 있지 지금 당장은 그게 큰 돈이 되...
3,2022.02.15 00:01:01,논톨가스쏜다 ㅋㅋㅋㅋㅋ
4,2022.02.15 00:01:05,가만히 생각해 봤는데 선물로 배율박는거보다 차라리 바카라에 박는게 더 이득일꺼같다어...
...,...,...
6391,2022.02.16 08:58:28,하 시발 기대댄다 지금 살짝살짝 내리는게 시 대불장 추진력이지
6392,2022.02.16 08:59:09,롱충보다 숏충이 많은 이유가 뭐야 대체 이유가 뭐지
6393,2022.02.16 08:59:17,엑시부활했다 일단 만원돌파가즈아 ㅇㅇ
6394,2022.02.16 08:59:19,일단 아무거나 사라 ㅋㅋ불장이다


## 전처리

In [145]:
def data_processing_stopword(files):
    m=Mecab()
    result=[]
    for file in files:
        rs=[]
        for i in tqdm(file['data']):
            tokenized_sentence =m.morphs(str(i))
            stopwords_removed_sentence=[word for word in tokenized_sentence if not word in stopwords]
            rs.append(stopwords_removed_sentence)
        result.append(rs)
    return result

In [146]:
test=data_processing_stopword(data)

100%|█████████████████████████████████████| 2848/2848 [00:00<00:00, 4687.16it/s]


In [220]:
test[9]

[['도지', '미', '쳣나', '끌어올리'],
 ['종목', '리플', '이클', '누사이퍼', '스테픈', '도지', '리플', '이클', '누사이퍼', '스테픈', '도지'],
 ['숏', '청산', '당했', '끌어올리', '작전'],
 ['형', '리플', '도지', '언', '넝', '언', '넝'],
 ['진짜',
  '도지',
  '세력',
  '심리',
  '마술사',
  '공포감',
  '개미',
  '떨구',
  '상승',
  '시키',
  '개미',
  '투자',
  '돈',
  '거나',
  '묶여',
  '거나',
  '너무',
  '오른',
  '같',
  '쉽',
  '잡',
  '못함'],
 ['도지', '같', '넘', '으면', '끝', '도지'],
 ['아파트', '란다', '공포', '버리', '껄'],
 ['도지', '주웠', '쫄리', '쫄', '린다', '쫄려'],
 ['대폭', '나오', '노', '프로', '짜리', '기대'],
 ['마지막', '설거지', '픽', '펑션', '엑스'],
 ['도시', '눈물', '머금', '손절', '쳤', '한잔', '니깐', '개떡', '상해', '노'],
 ['오르', '어딨', '노', '이기', '어제', '오늘', '이틀', '계속', '오르', '잖아', '오르'],
 ['도지사',
  '리플',
  '딸라',
  '저항',
  '못',
  '올라감',
  '도지',
  '까진',
  '무리',
  '없이',
  '갈',
  '리플',
  '지금',
  '저항라',
  '소송',
  '끝',
  '날',
  '까',
  '졍',
  '횡보',
  '친다',
  '서'],
 ['지금',
  '머스크',
  '욕먹',
  '열받',
  '언론',
  '개혁',
  '려니',
  '기득',
  '똘똘',
  '뭉쳐',
  '광고',
  '끊',
  '가짜',
  '뉴스',
  '공격',
  '함',
  '트위터',
  '유료',
  '서비스',
  '한다고',
  '물'

## 한글자는 제외

In [228]:
one_word_remove=[]
for result in test:
    temp=[]
    for word in result:
        tmp=[]
        for i in word:
            if len(i)>1:
                tmp.append(i)
        temp.append(tmp)
    one_word_remove.append(temp)

In [230]:
result=[]
for onefile in tqdm(one_word_remove):
    rs=[]
    for word in onefile:
        tmp=[]
        for i in word:
            if m.pos(i)[0][1]=='NNG' or m.pos(i)[0][1]=='NNP'or m.pos(i)[0][1]=='VCP' or m.pos(i)[0][1]=='VCN' or m.pos(i)[0][1]=='VV' or m.pos(i)[0][1]=='MAG' or m.pos(i)[0][1]=='VA':
                tmp.append(m.pos(i)[0][0])
        rs.append(tmp)
    result.append(rs)

100%|███████████████████████████████████████████| 10/10 [00:05<00:00,  1.71it/s]


## 3차원 배열 2차원으로 변경후 데이터프레임에 저장

In [231]:
sentence=[]
for file in result:
    sen=[]
    for word in file:
        result=' '.join(s for s in word)
        sen.append(result)
    sentence.append(sen)

In [232]:
for i in range(len(files)):
    files[i]['result']=sentence[i]

In [233]:
files[0]

,date,data,result
0,2022.08.13 00:01:35,앵커에 다시 다박고 잔다 내일이면 원은 가겠지,앵커 내일
1,2022.08.13 00:01:44,앵커는 퍼는 더떨어져 선물 숏때매 확 못내릴뿐 ㅇㅇ,앵커 선물
2,2022.08.13 00:03:18,빗썸도 잡코펌핑좀 하나 할때됬는데,펌핑
3,2022.08.13 00:04:02,롱 왤케 시원찮냐 ㅋㅋ흠,시원찮
4,2022.08.13 00:04:23,시 넘엇네,
...,...,...,...
2360,2022.08.14 08:56:38,경주마 하이브,경주마 하이브
2361,2022.08.14 08:57:08,이클 ㅈ된거같은데 ㅋㅋㅋㅋㅋㅋ 지금이라도 팔아라 ㅋㅋㅋㅋ,이클 지금 아라
2362,2022.08.14 08:57:50,시발 병신들아 코인 좀 떨어졌다고 울거나 그러는거 아니지 잠깐 눈물좀 닦고,코인 눈물
2363,2022.08.14 08:57:52,빗썸 콜라 다시 쏘냐,콜라


## 저장

In [199]:
file_names=os.listdir('/Users/home/고점')
for df in range(len(files)):
    try:
        files[df].to_csv('/Users/home/Test/'+file_names[df],encoding='utf-8')
    except:
        pass

## 빈도수 체크

In [203]:
len(sentence)

10

In [204]:
frequent=Counter(sentence[0]).most_common()
frequent

[('', 70),
 ('남자 심리 남 지난주 일요일 일요일 날 카페 남자 보 돌아 금요일 오늘 저녁 자 그날 일요일 카톡 이후 추가 연락 아예 생각 죠',
  8),
 ('이클', 8),
 ('엑', 5),
 ('메디 블록', 4),
 ('돔 황 챠', 4),
 ('이더리움', 3),
 ('솔라나', 3),
 ('스 트', 3),
 ('가즈', 3),
 ('서울 가장 존재감 잘 모르 투표', 3),
 ('첨 형 칭찬 뒤 론 연락 끊 버리 이유', 3),
 ('에이다', 3),
 ('여자 애 눈빛', 3),
 ('천만 가능', 2),
 ('산도깨비 친구 몸 좋 친구', 2),
 ('취 금', 2),
 ('황 클', 2),
 ('웨이브', 2),
 ('남자 코인 이캐시', 2),
 ('좇 로우', 2),
 ('스트라이크', 2),
 ('경주마 슬슬', 2),
 ('앵커', 2),
 ('대한민국 최고 렘 투표', 2),
 ('술 자리 경우 남자 동호회 술자리 대화 임 남자 형 술 마시 랑 클럽 가요 남자 체력 끝내 남자 가요 형 이랑 클럽 술값 형 계산 남자 클럽 남자 속마음',
  2),
 ('폴리곤', 2),
 ('먹', 2),
 ('존버 결과', 2),
 ('에이다 화이팅', 2),
 ('이클 숏', 2),
 ('아재 여자 쟁탈 아재 여자 쟁탈 남자 재산 여자 남자 거르 남자 사랑', 2),
 ('최근 기준 인기 코인 순위 최근 기준 인기 코인 순위 점수 기준 최근 커뮤니티 디 코인 블라인드 엠 언급 횟수', 2),
 ('비트코인 가격 억 미래', 2),
 ('와이프 와이프 랑 결혼 남자 많', 2),
 ('목표', 2),
 ('부정출발 항상 음전', 2),
 ('앵커 내일', 1),
 ('앵커 선물 숏 확 못', 1),
 ('빗 썸 잡 코 펌핑', 1),
 ('롱 시원찮 흠', 1),
 ('넘 엇', 1),
 ('속보 동대문 출발 가노', 1),
 ('숏 탓 이제', 1),
 ('피엠 매집 아직 매집 덜 쏜 꼬라박 버릴', 1),
 ('주말 무조건 반', 

### TF-IDF

In [216]:
corpus=list(files[9]['result'])

In [217]:
s=[]
for i in corpus:
    i=i.replace(" ","")
    s.append(i)

In [218]:
tfidfv=TfidfVectorizer(max_features=10).fit(s)
tfidfv.transform(corpus).toarray()
tfidfv.vocabulary_

{'썬썬': 6,
 '도지': 2,
 '그로스톨': 1,
 '폴리곤': 9,
 '솔라나': 5,
 '밸리디티낸': 4,
 '웨이브리브랜딩': 7,
 '가스': 0,
 '도지숏도지숏': 3,
 '케이코베상장': 8}